## Yawn Detector and Counting

In [ ]:
#importing libraries
import cv2
import dlib
import numpy as np

In [ ]:
PREDICTOR_PATH = "shape_predictor_68_face_landmarks.dat" #this is the crux of this project
predictor = dlib.shape_predictor(PREDICTOR_PATH)
detector = dlib.get_frontal_face_detector()

#### About "shape_predictor_68_face_landmarks.dat"

It's a pre-trained facial landmarks detector model. It is used to estimate the location of 68 coordinates (x, y) that map the facial points on a person’s face like image below.

![title](landmarks.PNG)

In [ ]:
#this function will return the x,y co ordinates of 68 landmarks
def get_landmarks(im):
    rects = detector(im, 1)

    if len(rects) > 1:
        return "error"
    if len(rects) == 0:
        return "error"
    
    return np.matrix([[p.x, p.y] for p in predictor(im, rects[0]).parts()])

In [ ]:
# to annotate the facial landmarks
def annotate_landmarks(im, landmarks):
    im = im.copy()
    for idx, point in enumerate(landmarks):
        pos = (point[0, 0], point[0, 1])
        cv2.putText(im, str(idx), pos,
                    fontFace=cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,
                    fontScale=0.4,
                    color=(0, 0, 255))
        cv2.circle(im, pos, 3, color=(0, 255, 255))
    return im

In [ ]:
#this function will return the mean value of upper lip
def upper_lip(landmarks):
    upper_lip_pts = []
    for i in range(50,53):
        upper_lip_pts.append(landmarks[i])
    for i in range(61,64):
        upper_lip_pts.append(landmarks[i])
    upper_lip_all_pts = np.squeeze(np.asarray(upper_lip_pts))
    upper_lip_mean = np.mean(upper_lip_pts, axis=0)
    return int(upper_lip_mean[:,1])

In [ ]:
#this function will return the mean value of bottom lip
def bottom_lip(landmarks):
    bottom_lip_pts = []
    for i in range(65,68):
        bottom_lip_pts.append(landmarks[i])
    for i in range(56,59):
        bottom_lip_pts.append(landmarks[i])
    bottom_lip_all_pts = np.squeeze(np.asarray(bottom_lip_pts))
    bottom_lip_mean = np.mean(bottom_lip_pts, axis=0)
    return int(bottom_lip_mean[:,1])

In [ ]:
# this function will return the distance between two lips
def mouth_open(image):
    landmarks = get_landmarks(image)
    
    if landmarks == "error":
        return image, 0
    
    image_with_landmarks = annotate_landmarks(image, landmarks)
    upper_lip_center = upper_lip(landmarks)
    bottom_lip_center = bottom_lip(landmarks)
    lip_distance = abs(upper_lip_center - bottom_lip_center)
    return image_with_landmarks, lip_distance

In [ ]:
cap = cv2.VideoCapture(0)
yawns = 0
yawn_status = False 

while True:
    ret, frame = cap.read()   
    image_landmarks, lip_distance = mouth_open(frame)
    
    prev_yawn_status = yawn_status  
    
    if lip_distance > 25:
        yawn_status = True 
        #shows the text
        cv2.putText(frame, "Subject is Yawning", (50,450),cv2.FONT_HERSHEY_COMPLEX, 1,(0,0,255),2)
        #yawn counter text
        output_text = " Yawn Count: " + str(yawns + 1)
        #show yawn counter text
        cv2.putText(frame, output_text, (50,50),cv2.FONT_HERSHEY_COMPLEX, 1,(0,255,127),2) 
    else:
        yawn_status = False 
         
    if prev_yawn_status == True and yawn_status == False:
        yawns += 1

    cv2.imshow('Live Landmarks', image_landmarks )
    cv2.imshow('Yawn Detection', frame )
    
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows() 